In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Common functions
from common_functions import reduce_mem_usage, remove_rare_records

In [2]:
train = pd.read_csv('data/train_data.csv')
test = pd.read_csv('data/test_data.csv')
train = reduce_mem_usage(train, verbose=True)
test = reduce_mem_usage(test, verbose=True)

C:\Users\leongkw\AppData\Local\Temp\ipykernel_15556\696475909.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('data/train_data.csv')
C:\Users\leongkw\AppData\Local\Temp\ipykernel_15556\696475909.py:2: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv('data/test_data.csv')


Mem. usage decreased to 17354.28 Mb (13.1% reduction)
Mem. usage decreased to 15201.62 Mb (12.9% reduction)


### 1. Initial Filtering 
(1) Get only win10.0.0.0 <br>
(2) Get only consumer devices <br>
(3) Remove virtual machines <br>
(4) Get only windows home edition

In [3]:
tr_filter_criteria = (train['OsVer'] == '10.0.0.0') & \
                     (train['SkuEdition'] == 'Home') & \
                     (train['Census_IsVirtualDevice'] != 1) & \
                     (train['Census_MDC2FormFactor'].isin(['Desktop', 'Notebook', 'AllInOne', 'PCOther', 'Convertible', \
                                                          'Detachable', 'SmallTablet', 'LargeTablet']))

t_filter_criteria = (test['OsVer'] == '10.0.0.0') & \
                    (test['SkuEdition'] == 'Home') & \
                    (test['Census_IsVirtualDevice'] != 1) & \
                    (test['Census_MDC2FormFactor'].isin(['Desktop', 'Notebook', 'AllInOne', 'PCOther', 'Convertible', \
                                                          'Detachable', 'SmallTablet', 'LargeTablet']))

train_filtered = train[tr_filter_criteria]
test_filtered = test[t_filter_criteria]

print('Train Pre-Filter:', train.shape)
print('Train Post-Filter:', train_filtered.shape)
print('Test Pre-Filter:', test.shape)
print('Test Post-Filter:', test_filtered.shape)

Train Pre-Filter: (8921483, 83)
Train Post-Filter: (5354415, 83)
Test Pre-Filter: (7853253, 82)
Test Post-Filter: (4416597, 82)


#### Check the target distribution after filtering 

In [4]:
# Seems more or less equal
print(train['HasDetections'].value_counts())
print(train_filtered['HasDetections'].value_counts())

HasDetections
0    4462591
1    4458892
Name: count, dtype: int64
HasDetections
0    2713221
1    2641194
Name: count, dtype: int64


### 2. Categorical Columns Re-Mapping

In [5]:
# Columns to re-map/check
obj_cols = []

for col in train_filtered.columns:
    if train_filtered[col].dtype == 'object':
        obj_cols.append(col)
        print(col)
    else:
        pass

MachineIdentifier
ProductName
EngineVersion
AppVersion
AvSigVersion
Platform
Processor
OsVer
OsPlatformSubRelease
OsBuildLab
SkuEdition
PuaMode
SmartScreen
Census_MDC2FormFactor
Census_DeviceFamily
Census_ProcessorClass
Census_PrimaryDiskTypeName
Census_ChassisTypeName
Census_PowerPlatformRoleName
Census_InternalBatteryType
Census_OSVersion
Census_OSArchitecture
Census_OSBranch
Census_OSEdition
Census_OSSkuName
Census_OSInstallTypeName
Census_OSWUAutoUpdateOptionsName
Census_GenuineStateName
Census_ActivationChannel
Census_FlightRing


In [6]:
def show_value_counts(col):
    print(train_filtered[col].value_counts())
    print(test_filtered[col].value_counts())
    
    
def remap_col(col, map_dict):
    train_filtered[col] = train_filtered[col].map(map_dict)
    test_filtered[col] = test_filtered[col].map(map_dict)

In [7]:
'''
MachineIdentifier -> Can ignore its just id
ProductName -> Can consider to remove the rare records
EngineVersion -> Can consider to remove the rare records
AppVersion -> Can consider to remove the rare records
AvSigVersion -> Can consider to remove the rare records
Platform -> Useless cause we filtered this to only Win10
Processor -> Can consider to remove the rare records (arm64)
OsVer -> Useless cause we filtered this to only Win10
OsPlatformSubRelease - Ok to keep
OsBuildLab -> Can consider to remove the rare records
SkuEdition -> Useless
PuaMode -> Lots of nulls
SmartScreen -> To clean
Census_MDC2FormFactor -> Ok
Census_DeviceFamily -> Can consider to remove the rare records
Census_ProcessorClass -> Ok
Census_PrimaryDiskTypeName -> Can combine unknown/unspecified
Census_ChassisTypeName -> To clean
Census_PowerPlatformRoleName -> To clean
Census_InternalBatteryType -> To clean
Census_OSVersion -> Can consider to remove the rare records
Census_OSArchitecture -> Can consider to remove the rare records
Census_OSBranch -> Can consider to remove the rare records
Census_OSEdition -> To clean (I think this is fine actually)
Census_OSSkuName -> To clean (I think this is fine) - This is actually same as OSEdition
Census_OSInstallTypeName -> To clean (I think this is fine)
Census_OSWUAutoUpdateOptionsName -> Ok
Census_GenuineStateName -> Ok
Census_ActivationChannel -> Ok
Census_FlightRing -> Can consider to remove the rare records
'''

for col in obj_cols:
    show_value_counts(col)

MachineIdentifier
000007905a28d863f6d0d597892cd692    1
aab22fbcacaf5c6f41627a2928236d78    1
aab26960bed9eb54fe20fef4213d70b0    1
aab2657c22e533935e5755a610cc8aec    1
aab262512c74b1a0cadcd47118b89824    1
                                   ..
5552cf8a5e2e2481a8fc6568d7b8fd01    1
5552cd8d33e5c1e304304109051cd43c    1
5552cbd1dbae465c429cebf5fedf0e6a    1
5552ca5bb20c1a7a6e7f034df151b9c0    1
ffffff75ba4f33d938ccfdb148b8ea16    1
Name: count, Length: 5354415, dtype: int64
MachineIdentifier
0000010489e3af074adeac69c53e555e    1
aa9bd49b67993a49e63a2a0bc826b08e    1
aa9ba8da8ae4177dd4cc49fbe32fce0a    1
aa9bac7b4fa13402f8f552d66e264109    1
aa9bb2ea2caeacec5fb8f6491bb0faaf    1
                                   ..
554c9b91516caf0c2f6ffca0e95a7d65    1
554c9cbc9d72d01775401a33838ad795    1
554c9d5fc404e6b56d1666889d450b94    1
554ca09623888fff4b6648f04f727a07    1
fffffbd305a90eb0f93ee4f30a39c736    1
Name: count, Length: 4416597, dtype: int64
ProductName
win8defender     5353904
mse  

Census_OSArchitecture
amd64    4974923
x86       379474
arm64         18
Name: count, dtype: int64
Census_OSArchitecture
amd64    4112138
x86       304449
arm64         10
Name: count, dtype: int64
Census_OSBranch
rs4_release                  2224866
rs3_release_svc_escrow        880847
rs3_release                   757667
rs2_release                   567398
rs1_release                   528360
th2_release_sec               192686
th2_release                    90148
th1_st1                        73092
th1                            28024
rs5_release                     5877
rs3_release_svc_escrow_im       3110
rs_prerelease_flt               1251
rs_prerelease                   1079
winblue_ltsb_escrow                5
winblue_ltsb                       2
rs1_release_sec                    1
win8_gdr                           1
win8_ldr                           1
Name: count, dtype: int64
Census_OSBranch
rs4_release                  2138692
rs3_release                   655932
rs2_

In [8]:
'''
Explanation of Values

1. &#x01;
Hexadecimal 0x01 (decimal 1):
This typically means enabled or on.
In the context of SmartScreen settings, it likely indicates that a specific feature or protection level is turned on.

2. &#x02;
Hexadecimal 0x02 (decimal 2):
This value might represent a different state or level of protection compared to 0x01.
It could be used to signify a specific mode, such as intermediate protection or a secondary feature being enabled.

3. 00000000
Hexadecimal 0x00000000 (decimal 0):
This typically means disabled or off.
In the context of SmartScreen settings, it likely indicates that a specific feature or protection level is turned off.
'''

smart_screen_remap = {'RequireAdmin'  : 'RequireAdmin',
                      'ExistsNotSet'  : 'ExistNotSet', 
                      'Off'           : 'Off',
                      'Warn'          : 'Warn',
                      'Prompt'        : 'Prompt',
                      'Block'         : 'Block',
                      'On'            : 'On',
                      'off'           : 'Off',
                      '&#x02;'        : 'On',
                      'on'            : 'On',
                      '&#x01;'        : 'On',
                      'requireadmin'  : 'RequireAdmin', 
                      'Promt'         : 'Prompt',
                      'OFF'           : 'Off',
                      'Enabled'       : 'On',
                      'prompt'        : 'Prompt',
                      'warn'          : 'Warn',
                      '00000000'      : 'Off',
                      'requireAdmin'  : 'RequireAdmin'
                     }

show_value_counts('SmartScreen')
remap_col('SmartScreen', smart_screen_remap)
show_value_counts('SmartScreen')

SmartScreen
RequireAdmin    2783655
ExistsNotSet     609395
Off               89139
Warn              74301
Prompt            18408
Block             14437
On                  717
off                 540
&#x02;              150
on                   43
&#x01;               31
requireadmin          3
Promt                 1
OFF                   1
Enabled               1
prompt                1
warn                  1
00000000              1
requireAdmin          1
Name: count, dtype: int64
SmartScreen
RequireAdmin    2113669
ExistsNotSet     320408
Off               71756
Warn              68262
Prompt            14909
Block             13339
On                  794
off                 773
&#x02;              115
on                   37
&#x01;               34
requireadmin          7
OFF                   3
0                     3
Promprt               1
warn                  1
of                    1
BLOCK                 1
Name: count, dtype: int64


C:\Users\leongkw\AppData\Local\Temp\ipykernel_15556\1324870777.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_filtered[col] = train_filtered[col].map(map_dict)
C:\Users\leongkw\AppData\Local\Temp\ipykernel_15556\1324870777.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_filtered[col] = test_filtered[col].map(map_dict)


SmartScreen
RequireAdmin    2783659
ExistNotSet      609395
Off               89681
Warn              74302
Prompt            18410
Block             14437
On                  942
Name: count, dtype: int64
SmartScreen
RequireAdmin    2113676
ExistNotSet      320408
Off               72532
Warn              68263
Prompt            14909
Block             13339
On                  980
Name: count, dtype: int64


In [9]:
chassis_type_remap = {'Notebook'               : 'Notebook',
                      'Desktop'                : 'Desktop',
                      'Laptop'                 : 'Notebook',
                      'Portable'               : 'Portable',
                      'AllinOne'               : 'AllInOne',
                      'Convertible'            : 'Convertible',
                      'Detachable'             : 'Detachable',
                      'UNKNOWN'                : 'Unknown',
                      'HandHeld'               : 'Handheld',
                      'Tablet'                 : 'Tablet',
                      'Other'                  : 'Other',
                      'MiniTower'              : 'TowerDesktop',
                      'LowProfileDesktop'      : 'Desktop',
                      'Unknown'                : 'Unknown',
                      'SpaceSaving'            : 'MiniDesktop',
                      'Tower'                  : 'TowerDesktop',
                      'MiniPC'                 : 'MiniDesktop',
                      'BusExpansionChassis'    : 'Chassis',
                      'SubNotebook'            : 'Portable',
                      'LunchBox'               : 'MiniDesktop',
                      'MainServerChassis'      : 'Chassis',
                      '30'                     : 'Unknown',
                      'StickPC'                : 'MiniDesktop',
                      '0'                      : 'Unknown',
                      'RackMountChassis'       : 'Chassis',
                      'SubChassis'             : 'Chassis',
                      'MultisystemChassis'     : 'Chassis',
                      '32'                     : 'Unknown',
                      '31'                     : 'Unknown',
                      '35'                     : 'Unknown',
                      '36'                     : 'Unknown',
                      'SealedCasePC'           : 'Desktop',
                      '82'                     : 'Unknown',
                      'DockingStation'         : 'Chassis',
                      '25'                     : 'Unknown',
                      '49'                     : 'Unknown',
                      'CompactPCI'             : 'MiniDesktop',
                      '88'                     : 'Unknown',
                      '44'                     : 'Unknown'
                     }

show_value_counts('Census_ChassisTypeName')
remap_col('Census_ChassisTypeName', chassis_type_remap)
show_value_counts('Census_ChassisTypeName')

Census_ChassisTypeName
Notebook               3742657
Desktop                 685334
Laptop                  334526
Portable                201799
AllinOne                149118
Convertible              71848
Detachable               43982
UNKNOWN                  39768
HandHeld                 34171
Tablet                   11005
Other                    10437
MiniTower                 9551
LowProfileDesktop         7379
Unknown                   4581
SpaceSaving               3726
Tower                      911
MiniPC                     705
BusExpansionChassis        700
SubNotebook                658
LunchBox                   565
MainServerChassis          550
30                         200
StickPC                    116
0                           64
RackMountChassis            11
SubChassis                  11
MultisystemChassis          10
32                           8
31                           7
35                           4
36                           3
SealedCasePC    

C:\Users\leongkw\AppData\Local\Temp\ipykernel_15556\1324870777.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_filtered[col] = train_filtered[col].map(map_dict)
C:\Users\leongkw\AppData\Local\Temp\ipykernel_15556\1324870777.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_filtered[col] = test_filtered[col].map(map_dict)


Census_ChassisTypeName
Notebook        4077183
Desktop          692715
Portable         202457
AllInOne         149118
Convertible       71848
Unknown           44640
Detachable        43982
Handheld          34171
Tablet            11005
TowerDesktop      10462
Other             10437
MiniDesktop        5113
Chassis            1283
Name: count, dtype: int64
Census_ChassisTypeName
Notebook        3341491
Desktop          594866
Portable         162729
AllInOne         122891
Convertible       66916
Detachable        36264
Unknown           30987
Handheld          26876
TowerDesktop       9769
Tablet             9020
Other              8593
MiniDesktop        4913
Chassis            1277
Name: count, dtype: int64


In [10]:
power_platform_remap = {'Mobile'              : 'Mobile',
                        'Desktop'             : 'Desktop',
                        'Slate'               : 'Slate',
                        'Workstation'         : 'Workstation',
                        'UNKNOWN'             : 'Unknown',
                        'SOHOServer'          : 'Small/Home Office Server',
                        'AppliancePC'         : 'Appliance',
                        'EnterpriseServer'    : 'Enterprise Server',
                        'PerformanceServer'   : 'Enterprise Server',
                        'Unspecified'         : 'Unknown'
                       }

show_value_counts('Census_PowerPlatformRoleName')
remap_col('Census_PowerPlatformRoleName', power_platform_remap)
show_value_counts('Census_PowerPlatformRoleName')

Census_PowerPlatformRoleName
Mobile               4211119
Desktop               742449
Slate                 352697
Workstation            28525
UNKNOWN                 9746
SOHOServer              6054
AppliancePC             3729
EnterpriseServer          91
PerformanceServer          5
Name: count, dtype: int64
Census_PowerPlatformRoleName
Mobile               3450897
Desktop               638102
Slate                 286597
Workstation            24842
UNKNOWN                 7454
SOHOServer              5680
AppliancePC             2920
EnterpriseServer         100
PerformanceServer          4
Unspecified                1
Name: count, dtype: int64


C:\Users\leongkw\AppData\Local\Temp\ipykernel_15556\1324870777.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_filtered[col] = train_filtered[col].map(map_dict)
C:\Users\leongkw\AppData\Local\Temp\ipykernel_15556\1324870777.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_filtered[col] = test_filtered[col].map(map_dict)


Census_PowerPlatformRoleName
Mobile                      4211119
Desktop                      742449
Slate                        352697
Workstation                   28525
Unknown                        9746
Small/Home Office Server       6054
Appliance                      3729
Enterprise Server                96
Name: count, dtype: int64
Census_PowerPlatformRoleName
Mobile                      3450897
Desktop                      638102
Slate                        286597
Workstation                   24842
Unknown                        7455
Small/Home Office Server       5680
Appliance                      2920
Enterprise Server               104
Name: count, dtype: int64


In [11]:
train_filtered[train_filtered['Census_InternalBatteryType'] == 'unkn']['HasDetections'].value_counts()

HasDetections
0    108
1    105
Name: count, dtype: int64

In [12]:
batt_type_remap = {'lion'     : 'Lithium Ion',
                   'li-i'     : 'Lithium Ion',
                   '#'        : 'Unknown', # About 50/50 for target
                   'lip'      : 'Lithium Polymer',
                   'liio'     : 'Lithium Ion',
                   'li p'     : 'Lithium Polymer',
                   'li'       : 'Lithium Ion',
                   'nimh'     : 'Nickel Metal Hydride',
                   'pbac'     : 'Lead Acid',
                   'real'     : 'Unknown',
                   'lgi0'     : 'Lithium Ion',
                   'unkn'     : 'Unknown', # About 50/50 for target
                   'lipo'     : 'Lithium Polymer',
                   '4cel'     : '4-Cell',
                   'lhp0'     : 'Lithium Ion',
                   'ithi'     : 'Lithium Ion',
                   'batt'     : 'Unknown',
                   'bad'      : 'Unknown',
                   'ca48'     : 'Unknown',
                   'lipp'     : 'Lithium Polymer',
                   'lit'      : 'Lithium Ion',
                   'ots0'     : 'Unknown',
                   'lai0'     : 'Unknown',
                   'ÿÿÿÿ'     : 'Unknown',
                   'li-p'     : 'Lithium Polymer',
                   'lgs0'     : 'Unknown',
                   '4lio'     : 'Lithium Ion',
                   'h00j'     : 'Unknown',
                   'pad0'     : 'Unknown',
                   'l&#TAB#' : 'Unknown',
                   '8'        : 'Unknown',
                   'ip'      : 'Unknown',
                   'ram'      : 'Unknown',
                   'lio'     : 'Lithium Ion',
                   '3500'     : 'Unknown',
                   'li?'      : 'Lithium Ion',
                   'ion'     : 'Lithium Ion',
                   'li-l'     : 'Lithium Ion',
                   '@i'     : 'Unknown',
                   'l'       : 'Unknown',
                   'lgl0'     : 'Unknown',
                   '˙˙˙'     : 'Unknown',
                   '0x0b'     : 'Unknown',
                   'sail'     : 'Unknown',
                   '0ts0'     : 'Unknown',
                   'liÿÿ'     : 'Unknown'
                  }

show_value_counts('Census_InternalBatteryType')
remap_col('Census_InternalBatteryType', batt_type_remap)
show_value_counts('Census_InternalBatteryType')

Census_InternalBatteryType
lion        1360272
li-i         164996
#             46646
lip           39510
liio          22411
li p           6929
li             4081
nimh           2434
pbac            986
real            599
lgi0            262
unkn            213
lipo            139
4cel            120
lhp0             98
ithi             70
batt             56
bad              16
ca48             16
lipp              7
lit               6
ots0              4
lai0              3
ÿÿÿÿ              3
li-p              3
lgs0              2
4lio              2
h00j              2
pad0              1
l&#TAB#          1
8                 1
ip               1
ram               1
lio              1
3500              1
li?               1
ion              1
li-l              1
@i              1
l                1
lgl0              1
˙˙˙              1
0x0b              1
sail              1
0ts0              1
liÿÿ              1
Name: count, dtype: int64
Census_InternalBatteryType


C:\Users\leongkw\AppData\Local\Temp\ipykernel_15556\1324870777.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_filtered[col] = train_filtered[col].map(map_dict)
C:\Users\leongkw\AppData\Local\Temp\ipykernel_15556\1324870777.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_filtered[col] = test_filtered[col].map(map_dict)


Census_InternalBatteryType
Lithium Ion             1552202
Unknown                   47574
Lithium Polymer           46588
Nickel Metal Hydride       2434
Lead Acid                   986
4-Cell                      120
Name: count, dtype: int64
Census_InternalBatteryType
Lithium Ion             1074565
Unknown                   38780
Lithium Polymer           33167
Nickel Metal Hydride       1896
Lead Acid                   692
4-Cell                       78
Name: count, dtype: int64


In [13]:
non_obj_cols = []

for col in train_filtered.columns:
    if col not in obj_cols:
        non_obj_cols.append(col)
        print(col)

IsBeta
RtpStateBitfield
IsSxsPassiveMode
DefaultBrowsersIdentifier
AVProductStatesIdentifier
AVProductsInstalled
AVProductsEnabled
HasTpm
CountryIdentifier
CityIdentifier
OrganizationIdentifier
GeoNameIdentifier
LocaleEnglishNameIdentifier
OsBuild
OsSuite
IsProtected
AutoSampleOptIn
SMode
IeVerIdentifier
Firewall
UacLuaenable
Census_OEMNameIdentifier
Census_OEMModelIdentifier
Census_ProcessorCoreCount
Census_ProcessorManufacturerIdentifier
Census_ProcessorModelIdentifier
Census_PrimaryDiskTotalCapacity
Census_SystemVolumeTotalCapacity
Census_HasOpticalDiskDrive
Census_TotalPhysicalRAM
Census_InternalPrimaryDiagonalDisplaySizeInInches
Census_InternalPrimaryDisplayResolutionHorizontal
Census_InternalPrimaryDisplayResolutionVertical
Census_InternalBatteryNumberOfCharges
Census_OSBuildNumber
Census_OSBuildRevision
Census_OSInstallLanguageIdentifier
Census_OSUILocaleIdentifier
Census_IsPortableOperatingSystem
Census_IsFlightingInternal
Census_IsFlightsDisabled
Census_ThresholdOptIn
Census_F

In [14]:
'''
IsBeta -> Can remove this column (Binary) V v skewed distribution
RtpStateBitfield -> Ok. (Categorical)
IsSxsPassiveMode -> Ok (Binary)
DefaultBrowsersIdentifier -> Can consider removing rare records
AVProductStatesIdentifier -> Can consider removing rare records
AVProductsInstalled -> Ok (Integer)
AVProductsEnabled -> Ok (Integer)
HasTpm -> Ok (Binary)
CountryIdentifier -> Can consider dimension reduction
CityIdentifier -> Can consider removing rare records
OrganizationIdentifier -> Can consider removing rare records
GeoNameIdentifier -> Can consider removing rare records
LocaleEnglishNameIdentifier -> Can consider removing rare records
OsBuild -> Can consider removing rare records
OsSuite -> Can remove this column -> V v skewed distribution
IsProtected -> Ok (Binary)
AutoSampleOptIn -> Can remove this column (Binary) V v skewed distribution
SMode -> Can remove this column (Binary) V v skewed distribution
IeVerIdentifier -> Can consider removing rare records
Firewall -> Ok (Binary)
UacLuaenable -> Can consider removing rare records 
Census_OEMNameIdentifier -> Can consider removing rare records
Census_OEMModelIdentifier -> Can consider removing rare records
Census_ProcessorCoreCount -> Can consider binning
Census_ProcessorManufacturerIdentifier -> Can consider removing rare records
Census_ProcessorModelIdentifier -> Can consider removing rare records
Census_PrimaryDiskTotalCapacity -> Can consider binning
Census_SystemVolumeTotalCapacity -> Can consider binning
Census_HasOpticalDiskDrive -> Binary
Census_TotalPhysicalRAM -> Can consider binning
Census_InternalPrimaryDiagonalDisplaySizeInInches -> Can consider binning
Census_InternalPrimaryDisplayResolutionHorizontal -> Can consider binning
Census_InternalPrimaryDisplayResolutionVertical -> Can consider binning
Census_InternalBatteryNumberOfCharges -> Can consider binning
Census_OSBuildNumber -> Can consider removing rare records
Census_OSBuildRevision -> Can consider removing rare records
Census_OSInstallLanguageIdentifier -> Ok
Census_OSUILocaleIdentifier -> Can consider removing rare records
Census_IsPortableOperatingSystem -> Can remove this column (Binary) V v skewed distribution
Census_IsFlightingInternal -> Can remove this column (Binary) V v skewed distribution
Census_IsFlightsDisabled -> Can remove this column (Binary) V v skewed distribution
Census_ThresholdOptIn -> Can remove this column (Binary) V v skewed distribution
Census_FirmwareManufacturerIdentifier -> Can consider removing rare records
Census_FirmwareVersionIdentifier -> Can consider removing rare records
Census_IsSecureBootEnabled -> Ok (Binary)
Census_IsWIMBootEnabled ->  Can remove this column (Binary) V v skewed distribution
Census_IsVirtualDevice -> Can remove this column (Binary)
Census_IsTouchEnabled -> Ok (Binary)
Census_IsPenCapable -> Ok (Binary)
Census_IsAlwaysOnAlwaysConnectedCapable -> Ok (Binary)
Wdft_IsGamer -> Ok (Binary)
Wdft_RegionIdentifier -> Ok
'''

for col in non_obj_cols:
    show_value_counts(col)

IsBeta
0    5354414
1          1
Name: count, dtype: int64
IsBeta
0    4416597
Name: count, dtype: int64
RtpStateBitfield
7.0     5185421
0.0      120043
5.0       14928
8.0       13430
3.0        2396
1.0         751
35.0         15
Name: count, dtype: int64
RtpStateBitfield
7.0     4280454
0.0       96939
8.0       12480
5.0        9044
3.0        1440
1.0         523
35.0         13
Name: count, dtype: int64
IsSxsPassiveMode
0    5248992
1     105423
Name: count, dtype: int64
IsSxsPassiveMode
0    4335802
1      80795
Name: count, dtype: int64
DefaultBrowsersIdentifier
3196.0    28256
239.0     26273
1632.0    24179
3176.0    19005
146.0     13641
          ...  
1783.0        1
383.0         1
993.0         1
2744.0        1
1149.0        1
Name: count, Length: 1329, dtype: int64
DefaultBrowsersIdentifier
239.0     22057
3196.0    19592
1632.0    13498
3176.0    10412
146.0      8734
          ...  
247.0         1
2308.0        1
2210.0        1
2616.0        1
391.0         1
Nam

Firewall
1.0    4352973
0.0      59333
Name: count, dtype: int64
UacLuaenable
1.0           5332693
0.0             15178
48.0               89
2.0                29
49.0               15
3.0                 4
6357062.0           2
16777216.0          1
5.0                 1
7798884.0           1
Name: count, dtype: int64
UacLuaenable
1.0            4397143
0.0              14826
48.0                83
2.0                 17
49.0                11
6357062.0            4
3.0                  4
808482880.0          1
537591872.0          1
Name: count, dtype: int64
Census_OEMNameIdentifier
2668.0    843163
2102.0    824046
585.0     682554
525.0     679770
2206.0    475322
           ...  
2040.0         1
696.0          1
5652.0         1
2660.0         1
4460.0         1
Name: count, Length: 1819, dtype: int64
Census_OEMNameIdentifier
2668.0    693990
2102.0    690469
525.0     572009
585.0     535586
1443.0    380922
           ...  
352.0          1
4352.0         1
316.0          1


Census_OSUILocaleIdentifier
31     1449887
34      521296
125     295972
30      246141
158     236128
        ...   
46           1
3            1
150          1
101          1
1            1
Name: count, Length: 126, dtype: int64
Census_IsPortableOperatingSystem
0    5352942
1       1473
Name: count, dtype: int64
Census_IsPortableOperatingSystem
0    4415112
1       1485
Name: count, dtype: int64
Census_IsFlightingInternal
0.0    829749
1.0         2
Name: count, dtype: int64
Census_IsFlightingInternal
0.0    609791
1.0         1
Name: count, dtype: int64
Census_IsFlightsDisabled
0.0    5331305
1.0         22
Name: count, dtype: int64
Census_IsFlightsDisabled
0.0    4393907
1.0          4
Name: count, dtype: int64
Census_ThresholdOptIn
0.0    1959967
1.0        175
Name: count, dtype: int64
Census_ThresholdOptIn
0.0    1358582
1.0        156
Name: count, dtype: int64
Census_FirmwareManufacturerIdentifier
142.0    1566659
554.0     977102
628.0     794417
556.0     639236
355.0     45

KeyError: 'HasDetections'

In [ ]:
disk_type_remap = {'HDD' : 'HDD',
                   'SSD' : 'SDD',
                   'UNKNOWN' : 'Unknown',
                   'Unspecified' : 'Unknown'
                  }

show_value_counts('Census_PrimaryDiskTypeName')
remap_col('Census_PrimaryDiskTypeName', disk_type_remap)
show_value_counts('Census_PrimaryDiskTypeName')

### 3. Remove the Rare Records (Optional) 

In [ ]:
# To consider
'''
ProductName -> Can consider to remove the rare records
EngineVersion -> Can consider to remove the rare records
AppVersion -> Can consider to remove the rare records
AvSigVersion -> Can consider to remove the rare records
Processor -> Can consider to remove the rare records (arm64)
OsBuildLab -> Can consider to remove the rare records
Census_DeviceFamily -> Can consider to remove the rare records
Census_OSVersion -> Can consider to remove the rare records
Census_OSArchitecture -> Can consider to remove the rare records
Census_OSBranch -> Can consider to remove the rare records
Census_FlightRing -> Can consider to remove the rare records
IsBeta -> Can remove this column (Binary) V v skewed distribution
DefaultBrowsersIdentifier -> Can consider removing rare records
AVProductStatesIdentifier -> Can consider removing rare records
CityIdentifier -> Can consider removing rare records
OrganizationIdentifier -> Can consider removing rare records
GeoNameIdentifier -> Can consider removing rare records
LocaleEnglishNameIdentifier -> Can consider removing rare records
OsBuild -> Can consider removing rare records
IeVerIdentifier -> Can consider removing rare records
UacLuaenable -> Can consider removing rare records 
Census_OEMNameIdentifier -> Can consider removing rare records
Census_OEMModelIdentifier -> Can consider removing rare records
Census_ProcessorCoreCount -> Can consider binning
Census_ProcessorManufacturerIdentifier -> Can consider removing rare records
Census_ProcessorModelIdentifier -> Can consider removing rare records
Census_OSBuildNumber -> Can consider removing rare records
Census_OSBuildRevision -> Can consider removing rare records
Census_OSUILocaleIdentifier -> Can consider removing rare records
Census_FirmwareManufacturerIdentifier -> Can consider removing rare records
Census_FirmwareVersionIdentifier -> Can consider removing rare records
'''

### 4. Binning (Optional) 

In [ ]:
# To consider
'''
Census_InternalPrimaryDiagonalDisplaySizeInInches -> Can consider binning
Census_InternalPrimaryDisplayResolutionHorizontal -> Can consider binning
Census_InternalPrimaryDisplayResolutionVertical -> Can consider binning
Census_InternalBatteryNumberOfCharges -> Can consider binning
Census_PrimaryDiskTotalCapacity -> Can consider binning
Census_SystemVolumeTotalCapacity -> Can consider binning
Census_ProcessorCoreCount -> Can consider binning
'''

### 5. To Remove (To Decide)
Criteria: <br>
(1) Either very skewed <br>
(2) Lots of Nulls <br>
(3) ID columns (useless)

In [ ]:
'''
# Uncontroversial
MachineIdentifier -> Can ignore its just id
Platform -> Useless cause we filtered this to only Win10
OsVer -> Useless cause we filtered this to only Win10
SkuEdition -> Useless
IsBeta -> Can remove this column (Binary) V v skewed distribution
OsSuite -> Can remove this column -> V v skewed distribution

# More Controversial
PuaMode -> Lots of nulls
AutoSampleOptIn -> Can remove this column (Binary) V v skewed distribution
SMode -> Can remove this column (Binary) V v skewed distribution
Census_IsPortableOperatingSystem -> Can remove this column (Binary) V v skewed distribution
Census_IsFlightingInternal -> Can remove this column (Binary) V v skewed distribution
Census_IsFlightsDisabled -> Can remove this column (Binary) V v skewed distribution
Census_ThresholdOptIn -> Can remove this column (Binary) V v skewed distribution
Census_IsWIMBootEnabled ->  Can remove this column (Binary) V v skewed distribution
'''

In [15]:
train_filtered.to_csv('train_filtered.csv', index=False)
test_filtered.to_csv('test_filtered.csv', index=False)